In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from multiprocessing import Pool
from src.mp_script import clusterKMean
from functools import partial

RandomState = 1

##### clutering delta gold and xp each team fight

In [2]:
if __name__ ==  '__main__':
    new_time_stat_df = pd.read_csv("./new_dataframe/new_time_stat_df.csv", index_col=0)
    new_teamfight_df = pd.read_csv("./new_dataframe/new_teamfight_df.csv", index_col=0)
    ##### calculate time before team fight and merge to team status on times
    new_teamfight_df['time_before_fight'] = new_teamfight_df['start'] - new_teamfight_df['start']%60
    new_teamfight_df = new_teamfight_df.drop(['start', 'end'], axis=1)
    teamfight_status = new_teamfight_df.merge(new_time_stat_df, left_on=['match_id', 'time_before_fight'], right_on=['match_id', 'times'])
    teamfight_status = teamfight_status.drop(['times'], axis=1)
    ##### scale for cluster
    scaler = MinMaxScaler()
    teamfight_cluster_df = teamfight_status[['radiant_total_delta_gold', 'dire_total_delta_gold', 'radiant_total_delta_xp', 'dire_total_delta_xp']]
    x_scale = scaler.fit_transform(teamfight_cluster_df.values)
    scale_df = pd.DataFrame(x_scale, columns=teamfight_cluster_df.columns)
    with Pool(processes=4) as p:
        output = p.map(partial(clusterKMean, scale_df), range(2,10,1))
        p.close()
        p.join()
        print('finish')
